In [1]:
import numpy as np
import numpy as np
from concrete import fhe
from Bio.Seq import Seq, MutableSeq

import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from concrete_biopython.FheSeq import FheSeq, FheMutableSeq
from concrete_biopython.SeqWrapper import SeqWrapper


In [2]:
fhe.zeros(0)

array([], dtype=int64)

In [3]:
seq1 = Seq('ACCAGGTAC')
seq2 = MutableSeq('CGTTAGC')
seq = seq1+seq2

In [12]:
def process_seq(s):
    s1=s[0:9]
    s2=s[9:]
    return s

In [5]:
output_seq = process_seq(seq)
print(output_seq)
print(seq)

ACCAGGTACCGTTAGC
ACCAGGTACCGTTAGC


In [6]:
# wrap a fhe circuit in order to input and output Bio.Seq objects.
def circuit_wrapper(circuit, seq, simulate=False):
    # convert Seq objects to integers with SeqWrapper.toIntegers
    integers = SeqWrapper(seq).toIntegers()
    # run the circuit with integer inputs
    integer_output = circuit.simulate(integers) if simulate else circuit.encrypt_run_decrypt(integers)
    # convert back the integer outputs into a Seq objects with SeqWrapper.toSeq
    return SeqWrapper(integer_output).toSeq()

In [7]:
def process_seq_adapter(integer_seq):
    # convert integer to FheSeq object, process it and return result
    return process_seq(FheSeq(integer_seq)).toArray()

In [10]:
# compile the process_seq_adapter function and create a circuit
compiler = fhe.Compiler(lambda data: process_seq_adapter(data), {"data": "encrypted"})
circuit = compiler.compile(
    inputset=[
    np.random.randint(0, SeqWrapper.maxInteger()+1, size=(len(seq),))
    for _ in range(100)
    ],
    configuration=fhe.Configuration(
        enable_unsafe_features=True,
        use_insecure_key_cache=True,
        insecure_key_cache_location=".keys",
        dataflow_parallelize=False, # setting it to True makes the jupyter kernel crash
    ),
    verbose=False,
)

In [11]:
# now we can run our circuit on Seq objects and compare the result with output_seq

# with simulation
fheSim_output_seq = circuit_wrapper(circuit, seq, True)
print('Simulated :', fheSim_output_seq)
assert(output_seq == fheSim_output_seq)

# and without (slower)


Simulated : ACCAGGTACCGTTAGC
